In [1]:
#Imports
import keras
import keras.utils
from keras import utils as np_utils
from sklearn import metrics
from sklearn.metrics import matthews_corrcoef


C:\Users\Hamish\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Function to load all the data
def load_all_data(filename):
    import numpy as np
    
    a = np.load(filename)
    d = (dict(zip(("data1{}".format(k) for k in a), (a[k] for k in a))))

    return d

In [3]:
#Load the data
filename = ("alldata.npz")
X = load_all_data(filename)['data1arr_0']
y = load_all_data(filename)['data1arr_1']

IDs = load_all_data(filename)['data1arr_2']

In [4]:
#Split Data into Train_Test
from  sklearn.model_selection import GroupShuffleSplit
import numpy as np
test_size = 0.2
gss = GroupShuffleSplit(n_splits = 1, test_size = 0.2)

for train,test in gss.split(X, y, IDs):
    X_train = X[train]
    y_train = y[train]
    IDs_train = IDs[train]
    
    X_test = X[test]
    y_test = y[test]
    IDs_test = IDs[test]
    

fileoutname = 'train_test_data'
np.savez(fileoutname,X_train, y_train, X_test, y_test,IDs_train,IDs_test)

In [5]:
#Split Data in Train2_Dev
gss = GroupShuffleSplit(1, test_size = 0.2)

for train,test in gss.split(X, y, IDs):
    X_train2 = X[train]
    y_train2 = y[train]
    IDs_train = IDs[train]
    
    X_dev = X[test]
    y_dev = y[test]
    IDs_test = IDs[test]

fileoutname = 'train_dev_data'
np.savez(fileoutname,X_train, y_train, X_test, y_test, IDs_train, IDs_test)

In [6]:
#Add dimensions for X in CNN and Convert y to Categorical
X_train2 = np.expand_dims(X_train2,axis=4)
y_train2 = keras.utils.to_categorical(y_train2,num_classes=2)
X_dev = np.expand_dims(X_dev,axis=4)
y_dev = keras.utils.to_categorical(y_dev,num_classes=2)

C:\Users\Hamish\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  
C:\Users\Hamish\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


In [7]:
#Build the CNN
def  simpleCNN(self, units = 10):
    
    import keras
    from keras.layers import Dense
    from keras.layers import Conv2D, MaxPooling2D
    from keras.layers import Flatten
    from keras.models import Model, Input
    
    inp =  Input(shape = self.shape[1:], name='inp')
    #layer 1
    x = Conv2D(units, kernel_size=(1,1), strides = (1,1), activation='relu', data_format='channels_last')(inp)  
    #layer 2
    x = Conv2D(units, kernel_size=(2,2), strides = (1,1), activation='relu', data_format='channels_last')(x)  
    #layer 3
    x = Conv2D(units, kernel_size=(2,2), strides = (1,1), activation='relu', data_format='channels_last')(x) 
    #layer 4
    x = Conv2D(units, kernel_size=(2,2), strides = (2,2), activation='relu', data_format='channels_last')(x) 
    #layer 5
    x = MaxPooling2D(pool_size=(2,2), strides = (2,2), padding='valid', data_format='channels_last')(x)  
    #layer 6
    x = Flatten()(x)
    #layer 7
    out = Dense(2, activation='softmax',name='out')(x)
    
    model = Model(inputs = inp, outputs = out)
    
    return model

In [8]:
#Assign CNN to model
model = simpleCNN(X_train2)

In [9]:
#Define custom metric - Matthews Correlation
y_pred = model.predict(X_train2)

import keras.backend as K

def matthews_correlation(y_train2, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos
    
    y_pos = K.round(K.clip(y_train2, 0, 1))
    y_neg = 1 - y_pos
    
    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)
    
    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)
    
    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    
    return numerator / (denominator + K.epsilon())


In [10]:
#Fit Data to Model
from keras.optimizers import Adamax
from keras import metrics
adamax = Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0); #3e-4; 2e-3 is a default.
model.compile(optimizer=adamax, loss='categorical_crossentropy', metrics=['acc',matthews_correlation])
model.fit(X_train2, y_train2, epochs=40, batch_size=32, verbose = 1, validation_data = (X_dev, y_dev))

Train on 8830 samples, validate on 2629 samples
Epoch 1/40
8830/8830 [==============================] - 24s 3ms/step - loss: 2.3994 - acc: 0.6622 - matthews_correlation: 0.3243 - val_loss: 1.9250 - val_acc: 0.7006 - val_matthews_correlation: 0.4013
Epoch 2/40
8830/8830 [==============================] - 24s 3ms/step - loss: 1.6460 - acc: 0.7164 - matthews_correlation: 0.4328 - val_loss: 1.3130 - val_acc: 0.7554 - val_matthews_correlation: 0.5108
Epoch 3/40
8830/8830 [==============================] - 24s 3ms/step - loss: 1.4737 - acc: 0.7322 - matthews_correlation: 0.4643 - val_loss: 1.2657 - val_acc: 0.7569 - val_matthews_correlation: 0.5139
Epoch 4/40
8830/8830 [==============================] - 24s 3ms/step - loss: 1.3699 - acc: 0.7376 - matthews_correlation: 0.4752 - val_loss: 1.2519 - val_acc: 0.7535 - val_matthews_correlation: 0.5070
Epoch 5/40
8830/8830 [==============================] - 24s 3ms/step - loss: 1.2933 - acc: 0.7486 - matthews_correlation: 0.4972 - val_loss: 1.1101 